A pretty raw processing file for the four datasets:
- Adult
- Default
- Acoustic
- Covtype

Download the datasets first. See *get_data.txt* for instructions.

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
cons = False

In [2]:
def bucketize_continuous_column(input_train_df,
                                continuous_column_name,
                                num_quantiles=None,
                                bins=None):
    assert (num_quantiles is None or bins is None)
    if num_quantiles is not None:
          train_quantized, bins_quantized = pd.qcut(
            input_train_df[continuous_column_name],
            num_quantiles,
            retbins=True,
            labels=False)
        input_train_df[continuous_column_name] = pd.cut(
            input_train_df[continuous_column_name], bins_quantized, 
            labels=False)
    elif bins is not None:
        input_train_df[continuous_column_name] = pd.cut(
            input_train_df[continuous_column_name], bins, labels=False)

In [3]:
### This is for credit default dataset ########
# 2 classes (yes, no), 2 groups (male, female)

dat = pd.read_csv("./default.csv", sep = ",")
dat = dat.dropna()

CONT_COLUMNS = [
    'LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 
    'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 
    'PAY_AMT5', 'PAY_AMT6'
]
for col in CONT_COLUMNS:
    bucketize_continuous_column(dat, col, num_quantiles = 4)

dummy = dat['SEX'].astype(str).str.join(sep='*').str.get_dummies(sep='*')
dummy.columns = ['SEX_' +str(col) for col in dummy.columns]
dat = dat.join(dummy)

# dat.rename(columns={'SEX':'group', 'Default':'label'}, inplace=True)
dat.rename(columns={'Default':'label'}, inplace=True)

dummy = dat['EDUCATION'].astype(str).str.join(sep='*').str.get_dummies(sep='*')
dummy.columns = ['EDUCATION_' +str(col) for col in dummy.columns]
dat = dat.join(dummy)

dummy = dat['MARRIAGE'].astype(str).str.join(sep='*').str.get_dummies(sep='*')
dummy.columns = ['MARRIAGE_' +str(col) for col in dummy.columns]
dat = dat.join(dummy)

dat.drop(['EDUCATION', 'MARRIAGE'],axis =1, inplace = True)

dat = dat[[c for c in dat if c not in ['label']] + ['label']]
    
extra = ['label']

col_names = [c for c in dat if c not in extra]

col_names_new = []
for i in range(len(col_names)):
    col_names_new.append('x_' + str(i))
    
name_dict = dict(zip(col_names, col_names_new))

dat.rename(columns=name_dict, inplace=True)

dat = dat.dropna()
dat = dat.reset_index(drop=True)

dat.to_csv('./default.csv', index = False)

dat

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_25,x_26,x_27,x_28,x_29,x_30,x_31,x_32,x_33,label
0,2,1.0,1.0,0,0,0,0,0,0,2.0,...,1,0,0,0,0,0,0,1,0,0
1,2,0.0,2.0,0,0,0,0,0,0,2.0,...,1,0,0,0,0,0,1,0,0,0
2,1,0.0,3.0,-1,0,-1,0,0,0,1.0,...,1,0,0,0,0,0,1,0,0,0
3,1,0.0,2.0,0,0,0,0,0,0,2.0,...,0,0,0,0,0,0,0,1,0,0
4,1,3.0,1.0,0,0,0,0,0,0,3.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15305,1,1.0,2.0,0,0,0,0,0,0,3.0,...,1,0,0,0,0,0,1,0,0,0
15306,1,1.0,2.0,0,-1,-1,0,0,0,0.0,...,0,0,0,0,0,0,0,1,0,0
15307,1,2.0,2.0,0,0,0,0,0,0,3.0,...,0,1,0,0,0,0,1,0,0,0
15308,1,1.0,2.0,1,-1,0,0,0,-1,0.0,...,0,1,0,0,0,0,1,0,0,1


In [4]:
del dat

In [5]:
### This is for adult dataset ########
# 2 classes (yes, no), 3 groups (black, white, asian)

dat = pd.read_csv("./adult.data", sep = ",", header = None)
dat_test = pd.read_csv("./adult.test", sep = ",", header = None)
dat = dat.append(dat_test)

dat = dat.replace(r'^\s*$', np.nan, regex=True)
dat = dat.dropna()

dat.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', \
               'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'label']

dat = dat.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

s = dat.native_country.value_counts()
dat = dat[dat.native_country.isin(s[s >= 100].index)]

dat = dat.replace("-", "_", regex=True)

CONT_COLUMNS = [
      'age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week', 'education_num'
]

bucketize_continuous_column(dat, 'age', num_quantiles=4)
bucketize_continuous_column(dat, 'fnlwgt', num_quantiles=4)
bucketize_continuous_column(
    dat, 'capital_gain', bins=[-1, 1, 4000, 10000, 100000])
bucketize_continuous_column(
    dat, 'capital_loss', bins=[-1, 1, 1800, 1950, 4500])
bucketize_continuous_column(
    dat, 'hours_per_week', bins=[0, 39, 41, 50, 100])
bucketize_continuous_column(
    dat, 'education_num', bins=[0, 8, 9, 11, 16])


cat_names = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

for cn in cat_names:
    dummy = dat[cn].str.get_dummies()
    dummy.columns = [cn + '_' + str(col) for col in dummy.columns]
    dat = pd.concat([dat, dummy], axis = 1)

dat.drop(cat_names,axis = 1, inplace = True)

dat = dat[[c for c in dat if c not in ['label']] + ['label']]

extra = ['label']

col_names = [c for c in dat if c not in extra]
col_names_new = []
for i in range(len(col_names)):
    col_names_new.append('x_' + str(i))
    
name_dict = dict(zip(col_names, col_names_new))

dat.rename(columns=name_dict, inplace=True)

dat = dat.reset_index(drop=True)

dat['label'] = (dat['label'].apply(
      lambda x: '>50K' in x)).astype(int)

dat = dat.dropna()
dat = dat.reset_index(drop=True)

dat.to_csv('./data/adult.csv', index = False)

dat

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_73,x_74,x_75,x_76,x_77,x_78,x_79,x_80,x_81,label
0,2.0,0.0,3,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3.0,0.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2.0,2.0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3.0,2.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,3.0,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47036,2.0,2.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
47037,3.0,3.0,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
47038,2.0,3.0,3,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
47039,2.0,0.0,3,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
del dat

In [7]:
def get_data(filename):
    data = load_svmlight_file(filename)
    X = data[0].todense()
    y = data[1]
    return X, y

In [8]:
X, y = get_data(".data/acoustic_scale")

col_names_new = []
for i in range(X.shape[1]):
    col_names_new.append('x_' + str(i))

dat = pd.DataFrame(data = X, columns = col_names_new)
dat['label'] = y
dat['label'] = dat['label'] - 1

dat = dat.dropna()
dat = dat.reset_index(drop=True)

dat.to_csv('./data/acoustic.csv', index = False)

dat

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_41,x_42,x_43,x_44,x_45,x_46,x_47,x_48,x_49,label
0,-0.801621,-0.358074,-0.490782,-0.838876,-0.818791,-0.855470,-0.970358,-0.894983,-0.975610,-0.794006,...,-0.999436,-0.998703,-0.998950,-0.998843,-0.998764,-0.998018,-0.999088,-0.998534,-0.999080,1.0
1,-0.465054,0.004827,-0.644262,-0.964797,-0.926509,-0.965544,-0.959299,-0.952552,-0.989917,-0.980113,...,-0.998814,-0.997422,-0.997758,-0.997016,-0.996085,-0.996344,-0.997497,-0.997323,-0.997504,2.0
2,-0.937873,-0.983283,-0.926840,-0.983147,0.145058,-0.875283,-0.991453,-0.866716,-0.772981,-0.751429,...,-0.998759,-0.997926,-0.997475,-0.997775,-0.997955,-0.997033,-0.997419,-0.997110,-0.997149,2.0
3,-0.991044,-0.985472,-0.919018,-0.851009,-0.824401,-0.908792,-0.870483,-0.768306,-0.945808,-0.853743,...,-0.988946,-0.915846,-0.983169,-0.981640,-0.975182,-0.973862,-0.984878,-0.989552,-0.944301,1.0
4,-0.835440,-0.960212,-0.986046,-0.777397,0.145013,-0.868432,-0.970739,-0.986265,-0.937116,-0.960560,...,-0.989717,-0.977853,-0.953680,-0.966162,-0.976723,-0.971777,-0.980846,-0.963753,-0.931071,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78818,-0.982159,-0.991705,-0.986344,-0.941934,-0.855081,0.657359,-0.907114,-0.992991,-0.977500,-0.982855,...,-0.999752,-0.999152,-0.999557,-0.999002,-0.999150,-0.999118,-0.999036,-0.999091,-0.999163,1.0
78819,-0.996169,-0.988869,-0.611754,-0.964908,-0.976117,-0.636741,-0.965796,-0.778469,-0.880166,-0.952131,...,-0.985280,-0.965354,-0.959061,-0.928236,-0.961563,-0.950467,-0.816524,-0.866919,-0.962859,0.0
78820,-0.993963,-0.944042,-0.965695,-0.961683,-0.988961,-0.886123,-0.874592,0.524105,-0.950123,-0.975856,...,-0.996766,-0.974698,-0.998665,-0.989314,-0.998218,-0.991428,-0.995670,-0.994720,-0.996950,1.0
78821,-0.580417,-0.229012,-0.322154,-0.953214,-0.819430,-0.993652,-0.976006,-0.965645,-0.997137,-0.970819,...,-0.999384,-0.998576,-0.999103,-0.999199,-0.998337,-0.998814,-0.998878,-0.998900,-0.998766,2.0


In [9]:
del dat

In [10]:
X, y = get_data("./data/covtype.scale01")

col_names_new = []
for i in range(X.shape[1]):
    col_names_new.append('x_' + str(i))

dat = pd.DataFrame(data = X, columns = col_names_new)
dat['label'] = y
dat['label'] = dat['label'] - 1

dat = dat[dat['label'].isin([0, 1, 2, 5, 6])] # removing 3 and 4 classes

dat['label'] = dat['label'].replace(5, 3)
dat['label'] = dat['label'].replace(6, 4)

dat = dat.drop(dat.query('label == 0').sample(frac=.5).index)
dat = dat.drop(dat.query('label == 1').sample(frac=.5).index)

dat = dat.dropna()
dat = dat.reset_index(drop=True)

dat.to_csv('./data/covtype.csv', index = False)

dat

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_45,x_46,x_47,x_48,x_49,x_50,x_51,x_52,x_53,label
0,0.472736,0.386111,0.136364,0.191840,0.307494,0.446817,0.921260,0.937008,0.531496,0.853339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.441721,0.372222,0.333333,0.107373,0.312661,0.451735,0.976378,0.881890,0.362205,0.849157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.551776,0.411111,0.242424,0.231210,0.253230,0.831249,0.944882,0.929134,0.472441,0.473303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.476238,0.375000,0.015152,0.151754,0.262274,0.515667,0.866142,0.937008,0.606299,0.786700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.520760,0.125000,0.287879,0.173228,0.249354,0.730504,0.870079,0.767717,0.393701,0.573679,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321197,0.268634,0.425000,0.303030,0.060845,0.245478,0.015175,0.944882,0.933071,0.464567,0.116688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
321198,0.266133,0.422222,0.287879,0.047960,0.239018,0.013348,0.944882,0.933071,0.468504,0.117803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
321199,0.263632,0.441667,0.257576,0.042949,0.232558,0.012646,0.929134,0.948819,0.511811,0.119058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
321200,0.262631,0.472222,0.227273,0.042949,0.229974,0.012646,0.905512,0.964567,0.562992,0.120452,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
